In [55]:
import numpy as np
import pandas as pd

In [56]:
bill_summaries = pd.read_csv('../data/bill_summaries.csv')
bioinfo = pd.read_csv('../data/bioinfo.csv')
fec_ids = pd.read_csv('../data/fec_ids.csv')
ideology = pd.read_csv('../data/ideology.csv')
sponsored_legislation = pd.read_csv('../data/sponsored_legislation.csv')
terms = pd.read_csv('../data/terms.csv')
vote_compare = pd.read_csv('../data/vote_compare.csv')
contrib = pd.read_csv('../data/contrib.csv')



1. Do each of the CSVs have a primary key? If so, what?

In [57]:
bill_summaries.columns


Index(['actionDate', 'actionDesc', 'currentChamber', 'currentChamberCode',
       'lastSummaryUpdateDate', 'text', 'updateDate', 'versionCode',
       'bill.congress', 'bill.number', 'bill.originChamber',
       'bill.originChamberCode', 'bill.title', 'bill.type',
       'bill.updateDateIncludingText', 'bill.url'],
      dtype='object')

In [58]:
bill_summaries[['bill.type', 'bill.number', 'versionCode']].duplicated().value_counts()

False    2751
Name: count, dtype: int64

Primary key:

In [59]:

bioinfo.columns


Index(['bioguide_id', 'Full name', 'Chamber', 'State', 'Party', 'District',
       'birthYear', 'image', 'Office address', 'Phone', 'Website'],
      dtype='object')

In [60]:
bioinfo[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [61]:

fec_ids.columns


Index(['bioguide_id', 'fec_id'], dtype='object')

In [62]:
fec_ids[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [63]:

ideology.columns


Index(['bioname', 'chamber', 'left_right_ideology', 'state_abbrev',
       'district_code', 'icpsr', 'bioguide_id', 'party'],
      dtype='object')

In [64]:
ideology[['bioguide_id']].duplicated().value_counts()

False    545
Name: count, dtype: int64

In [65]:

sponsored_legislation.columns


Index(['introducedDate', 'type', 'url', 'number', 'title', 'bioguide_id'], dtype='object')

In [66]:
sponsored_legislation[['url']].duplicated().value_counts()

False    14379
Name: count, dtype: int64

In [67]:
terms.columns

Index(['bioguide_id', 'chamber', 'congress', 'stateCode', 'startYear',
       'endYear', 'district'],
      dtype='object')

In [68]:
terms[['bioguide_id', 'chamber', 'congress']].duplicated().value_counts()

False    3257
Name: count, dtype: int64

In [69]:

vote_compare.columns

Index(['bioname', 'comparison_member', 'agree'], dtype='object')

In [70]:
vote_compare[['bioname', 'comparison_member']].duplicated().value_counts()

False    206040
Name: count, dtype: int64

In [71]:
contrib.columns

Index(['contributor_name', 'contributor_aggregate_ytd', 'memo_text', 'pdf_url',
       'fec_committee_id', 'fec_id'],
      dtype='object')

In [72]:
contrib[['pdf_url']].duplicated().value_counts()

True     431028
False    217665
Name: count, dtype: int64

In [73]:
contrib.loc[contrib.duplicated('pdf_url', keep=False)].sort_values('pdf_url').head(10)['pdf_url'][600174]

'http://docquery.fec.gov/cgi-bin/fecimg/?10930084691'

In [74]:
contrib = contrib.drop_duplicates()

In [75]:
contrib[['pdf_url']].duplicated().value_counts()

True     378715
False    217665
Name: count, dtype: int64

2. Do any of the tables need to become multiple tables, or combined into one table?

Should be combined into one table (share the same primary key): bioinfo, fec_ids, ideology 

In [76]:
members = pd.merge(bioinfo, fec_ids,
                   on = 'bioguide_id',
                   how = 'outer',
                   validate = 'one_to_one',
                   indicator = 'matched')

In [77]:
members['matched'].value_counts()

matched
both          545
left_only       0
right_only      0
Name: count, dtype: int64

In [78]:
members = members.drop('matched', axis=1)

In [79]:
members = pd.merge(members, ideology,
                   on = 'bioguide_id',
                   how = 'outer',
                   validate = 'one_to_one',
                   indicator = 'matched')

In [80]:
members['matched'].value_counts()

matched
both          545
left_only       0
right_only      0
Name: count, dtype: int64

In [81]:
members = members.drop('matched', axis=1)

In [82]:
members.columns

Index(['bioguide_id', 'Full name', 'Chamber', 'State', 'Party', 'District',
       'birthYear', 'image', 'Office address', 'Phone', 'Website', 'fec_id',
       'bioname', 'chamber', 'left_right_ideology', 'state_abbrev',
       'district_code', 'icpsr', 'party'],
      dtype='object')

In [83]:
members.head(3).T

,0,1,2
bioguide_id,A000055,A000148,A000369
Full name,Robert B. Aderholt,Jake Auchincloss,Mark E. Amodei
Chamber,House of Representatives,House of Representatives,House of Representatives
State,Alabama,Massachusetts,Nevada
Party,Republican,Democratic,Republican
District,4.0,4.0,2.0
birthYear,1965.0,1988.0,1958.0
image,https://www.congress.gov/img/member/a000055_20...,https://www.congress.gov/img/member/67817e391f...,https://www.congress.gov/img/member/a000369_20...
Office address,"272 Cannon House Office Building, Washington, ...","1524 Longworth House Office Building, Washingt...","104 Cannon House Office Building, Washington, ..."
Phone,(202) 225-4876,(202) 225-5931,(202) 225-6155


In [84]:
members = members.drop(['Chamber', 'Party', 'District', 'State'], axis=1)


In [85]:
members.columns = [c.lower().replace(' ', '_') for c in members.columns]

In [86]:
members.head(3).T

,0,1,2
bioguide_id,A000055,A000148,A000369
full_name,Robert B. Aderholt,Jake Auchincloss,Mark E. Amodei
birthyear,1965.0,1988.0,1958.0
image,https://www.congress.gov/img/member/a000055_20...,https://www.congress.gov/img/member/67817e391f...,https://www.congress.gov/img/member/a000369_20...
office_address,"272 Cannon House Office Building, Washington, ...","1524 Longworth House Office Building, Washingt...","104 Cannon House Office Building, Washington, ..."
phone,(202) 225-4876,(202) 225-5931,(202) 225-6155
website,https://aderholt.house.gov/,https://auchincloss.house.gov,https://amodei.house.gov
fec_id,H6AL04098,H0MA04192,H2NV02395
bioname,"ADERHOLT, Robert","AUCHINCLOSS, Jake","AMODEI, Mark E."
chamber,House,House,House


In [87]:
members.to_csv('../data/thirdNF/members.csv', index=False)

In [88]:
bill_summaries.head(3).T

,0,1,2
actionDate,2025-10-15,2025-10-08,2025-10-08
actionDesc,Introduced in Senate,Introduced in House,Introduced in House
currentChamber,Senate,House,House
currentChamberCode,S,H,H
lastSummaryUpdateDate,2025-10-20T19:23:02Z,2025-10-20T14:56:29Z,2025-10-20T14:06:32Z
text,<p><strong>Shutdown Fairness Act</strong></p><...,<p><strong>Federal Worker Childcare Protection...,<p>This bill requires the federal government t...
updateDate,2025-10-20T19:23:15Z,2025-10-20T14:56:58Z,2025-10-20T14:07:00Z
versionCode,0,0,0
bill.congress,119,119,119
bill.number,3012,5720,5705


In [89]:
for c in bill_summaries.columns:
        print(c)
        d = bill_summaries.groupby(['bill.type', 'bill.number']).agg({c: 'nunique'})
        print(np.mean(d[c]) == 1)

actionDate
False
actionDesc
False
currentChamber
False
currentChamberCode
False
lastSummaryUpdateDate
False
text
False
updateDate
False
versionCode
False
bill.congress
True
bill.number
True
bill.originChamber
True
bill.originChamberCode
True
bill.title
True
bill.type
True
bill.updateDateIncludingText
True
bill.url
True


In [90]:
bills = bill_summaries[['bill.type', 'bill.number', 'bill.congress', 'bill.originChamber', 
                       'bill.originChamberCode', 'bill.title', 'bill.updateDateIncludingText',
                       'bill.url']].drop_duplicates()
bills.columns = [c.lower().replace('.', '_') for c in bills.columns]
bills.to_csv('../data/thirdNF/bills.csv')


In [91]:
bill_versions = bill_summaries.drop(['bill.congress', 'bill.originChamber', 
                       'bill.originChamberCode', 'bill.title', 'bill.updateDateIncludingText',
                       'bill.url'], axis=1)

In [92]:
bill_versions .columns = [c.lower().replace('.', '_') for c in bill_versions.columns]
bill_versions.to_csv('../data/thirdNF/bill_versions.csv')

In [93]:
vote_compare.head(3).T

,0,1,2
bioname,"GRASSLEY, Charles Ernest","GRASSLEY, Charles Ernest","GRASSLEY, Charles Ernest"
comparison_member,"MARKEY, Edward John","SCHUMER, Charles Ellis (Chuck)","WYDEN, Ronald Lee"
agree,0.021053,0.07193,0.052632


In [94]:
vote_compare.to_csv('../data/thirdNF/vote_compare.csv', index=False)